In [10]:
# imports
import os
import re
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
# initialize driver
def init_driver(headless=False):
    service = Service(executable_path="geckodriver/geckodriver.exe")
    options = Options()
    options.add_argument("--headless") if headless else None
    driver = webdriver.Firefox(service=service, options=options)
    driver.implicitly_wait(10) # seconds
    return driver

In [12]:
def search_auction_house(auction_house_name: str, url: str="https://hibid.com/companysearch", headless = False) -> dict[str, dict[str, str]]:
    """
    Search for an auction house on the specified URL and return its details.
    
    Parameters:
    - driver (WebDriver): The Selenium web driver instance.
    - auction_house_name (str): The name of the auction house to search for.
    - url (str, optional): The URL of the auction house search page. Default is "https://hibid.com/companysearch".
    
    Returns:
    Dict[str, Dict[str, str]]: A dictionary containing details about the found auction houses. The structure is:
    {
        'auction_house_name': {
            'id': 'auction_house_id',
            'href': 'auction_house_url',
            'location': 'auction_house_location'
        },
        ...
    }
    """
    driver = init_driver(headless=headless)
    driver.get(url)
    
    search_input = driver.find_element(By.CLASS_NAME, "search-input")
    search_input.find_element(By.TAG_NAME, "input").send_keys(auction_house_name)
    search_input.find_element(By.CLASS_NAME, "btn-primary").click()
    
    table_body = driver.find_element(By.CSS_SELECTOR, "#companySearch tbody")
    auction_house_rows = table_body.find_elements(By.TAG_NAME, "tr")
    
    auction_houses = {}
    
    for auction_house_row in auction_house_rows:
        name_elem = auction_house_row.find_element(By.CSS_SELECTOR, "td a")
        location_elem = auction_house_row.find_element(By.CSS_SELECTOR, "td:nth-child(2) a")
            
        auction_house_name = name_elem.text
        auction_house_href = name_elem.get_attribute("href")
        auction_house_id = auction_house_href.split("/")[-2]
        auction_house_location = location_elem.get_attribute("href")
        
        auction_houses[auction_house_name] = {
            "id": auction_house_id, 
            "href": auction_house_href, 
            "location": auction_house_location
        }
    
    driver.quit()

    return auction_houses


In [13]:
def extract_auction_details(auction_header):
    """Extract details for a single auction."""
    
    auction_title = auction_header.find_element(By.CLASS_NAME, "auction-title").text

    date_range_text = auction_header.find_element(By.XPATH, ".//p[contains(text(), 'Date(s)')]").text
    start_date_text, end_date_text = date_range_text.replace("Date(s) ", "").split(" - ")
    start_date = datetime.strptime(start_date_text, '%m/%d/%Y').date()
    end_date = datetime.strptime(end_date_text, '%m/%d/%Y').date()

    auction_type = auction_header.find_element(By.CSS_SELECTOR, ".bid-type.ng-star-inserted").text

    auction_status = auction_header.find_element(By.TAG_NAME,"app-auction-status")
    bidding_status = auction_status.find_element(By.TAG_NAME, "span").text
    is_bidding_open = True if "Open" in bidding_status else False

    shipping_status_element = auction_header.find_element(By.CSS_SELECTOR, ".shipping-type.text-primary.ng-star-inserted")
    shipping_status = "Not" not in shipping_status_element.text if shipping_status_element else False

    auction_links, lots = extract_auction_links_and_lots(auction_header)

    return {
        "auction_title": auction_title,
        "dates": {
            'start': start_date,
            'end': end_date
        },
        "auction_type": auction_type,
        "is_bidding_open": is_bidding_open,
        "shipping_available": shipping_status,
        "links": auction_links,
        "number_of_lots": lots
    }


def extract_auction_links_and_lots(auction_header):
    """Extract auction links and number of lots for a single auction."""
    
    auction_buttons = auction_header.find_elements(By.CSS_SELECTOR, ".auction-btn.btn.btn-primary.btn-block.ng-star-inserted")
    auction_links = {}
    lots = None

    for btn in auction_buttons:
        text = btn.text.strip()
        link = btn.get_attribute('href')
        if "Auction Details" in text:
            auction_links["register"] = link
        elif "View Catalog" in text:
            auction_links["catalog"] = link
            lots_text = btn.find_element(By.TAG_NAME, "span").text
            lots = re.search(r'\((\d+) [Ll][Oo][Tt][Ss]\)', lots_text)
            lots = int(lots.group(1)) if lots else None

    return auction_links, lots


def get_auction_company_details_and_listings(url="https://hibid.com/company/92115/encore-auctions-inc", headless=False):
    """
    Fetch the details of an auction company from the specified URL.

    This function scrapes and returns details about the auctioneer as well as information about each auction listing. 
    The auction information includes title, date range, type, bidding status, shipping availability, associated links, 
    and the number of lots.

    Parameters:
    - driver (webdriver object): The Selenium WebDriver object to interact with the browser.
    - url (str, optional): The URL of the auction company to scrape. Defaults to Encore Auctions Inc on HiBid.

    Returns:
    - dict: 
        - "auctioneer_details": A string containing details about the auctioneer.
        - "auctions": A dictionary where:
            - key (str): The title of the auction.
            - value (dict): A dictionary containing:
                - "dates": A dictionary with 'start' and 'end' dates of the auction.
                - "number_of_lots": An integer indicating the number of lots in the auction (if available).
                - "auction_type": A string describing the type of the auction.
                - "is_bidding_open": A boolean indicating if bidding is currently open.
                - "shipping_available": A boolean indicating if shipping is available for the auction.
                - "links": A dictionary containing links associated with the auction. Possible keys include "register" and "catalog".

    Example usage:
    ```
    from selenium import webdriver

    driver = webdriver.Chrome()
    result = fetch_auction_company(driver)
    print(result)
    driver.quit()
    ```
    """
    driver = init_driver()
    driver.get(url)
    
    auctioneer_details = driver.find_element(By.CLASS_NAME, "auctioneer-details").text
    
    auction_headers = driver.find_elements(By.TAG_NAME, "app-auction-header")

    all_auctions = {"auctioneer_details": auctioneer_details, "auctions": {}}
    
    for auction_header in auction_headers:
        details = extract_auction_details(auction_header)
        all_auctions["auctions"][details["auction_title"]] = {
            key: value for key, value in details.items() if key != "auction_title"
        }
    
    driver.quit()

    return all_auctions


In [14]:

def scrape_detailed_auction_data(driver):
    """
    Extract detailed auction data from the given webpage.

    Parameters:
    - driver (webdriver instance): An active Selenium webdriver instance.

    Returns:
    - list[dict]: A list of dictionaries containing details of each auction element.

    Each dictionary has the following structure:
    {
        "link": <auction link>,
        "title": <auction title>,
        "lot_number": <lot number>,
        "number_of_bids": <number of bids made>,
        "currency": <currency of the bid>,
        "max_bid": <highest bid amount>
    }
    """
    auction_elements = driver.find_elements(By.CSS_SELECTOR, 'app-lot-tile')
    print(f"Number of detailed auctions: {len(auction_elements)}")    

    auctions_data = []
    for auction in auction_elements:
        auction_dict = {}
        
        try:
            auction_dict["link"] = auction.find_element(By.CSS_SELECTOR, '.lot-link.lot-preview-link.link').get_attribute('href')
            auction_dict["title"] = auction.find_element(By.CSS_SELECTOR, 'h2.lot-title').text
            auction_dict["lot_number"] = auction.find_element(By.CSS_SELECTOR, '.text-primary.fw-bold').text.split(' ')[1]
            auction_dict["number_of_bids"] = int(auction.find_element(By.CSS_SELECTOR, '.lot-bid-history.btn-link').text.split(' ')[0])
            max_bid = auction.find_element(By.CSS_SELECTOR, '.lot-high-bid.font-weight-bold').text.split(':')[1].strip().split()
            auction_dict['currency'] = max_bid[1]
            auction_dict["max_bid"] = float(max_bid[0].strip())
        except Exception as e:
            print(f"Error extracting data for one of the auctions: {e}")

        auctions_data.append(auction_dict)

    
    return auctions_data

def scrape_condensed_auction_data(driver):
    """
    Extract condensed auction data from the given webpage.

    Parameters:
    - driver (webdriver instance): An active Selenium webdriver instance.

    Returns:
    - list[dict]: A list of dictionaries containing details of each condensed auction element.

    Each dictionary has the following structure:
    {
        "link": <auction link>,
        "title": <auction title>,
        "lot_number": <lot number>,
        "number_of_bids": <number of bids made>,
        "max_bid": <highest bid amount>
    }
    """
    auction_elements = driver.find_elements(By.TAG_NAME, 'app-lot-condensed')
    print(f"Number of condensed auctions: {len(auction_elements)}")

    auctions_data = []
    for auction in auction_elements:    
        auction_dict = {}
        
        try:
            auction_dict["link"] = auction.find_element(By.CSS_SELECTOR, '.lot-link').get_attribute('href')
            auction_dict["title"] = auction.find_element(By.CSS_SELECTOR, 'h2.lot-title').text
            auction_dict["lot_number"] = auction.find_element(By.TAG_NAME, 'strong').text.split(' ')[1]
            auction_dict["number_of_bids"] = int(auction.find_element(By.CSS_SELECTOR, '.lot-bid-history.btn-link').text.split(' ')[0])
            max_bid = auction.find_element(By.CSS_SELECTOR, 'dd.lot-high-bid').text.strip().split()
            auction_dict['currency'] = max_bid[1]
            auction_dict["max_bid"] = float(max_bid[0].strip())
        except Exception as e:
            print(f"Error extracting data for one of the auctions: {e}")

        auctions_data.append(auction_dict)

    
    return auctions_data


def scrape_auction_data(url, headless=False):
    """
    Extracts both detailed and condensed auction data from the given webpage.

    Parameters:
    - url (str): URL of the webpage containing auction elements (both detailed and condensed).

    Returns:
    - list[dict]: A list of dictionaries containing details of each auction element 
                  (combining both detailed and condensed auctions).
    Each dictionary has the following structure:
    {
        "link": <auction link>,
        "title": <auction title>,
        "lot_number": <lot number>,
        "number_of_bids": <number of bids made>,
        "currency": <currency of the bid>,
        "max_bid": <highest bid amount>
    }

    Each dictionary has the structure as described in the previous functions.
    """
    driver = init_driver(headless=headless)
    driver.get(url)
    time.sleep(5)
    # Wait for the elements to load or add an explicit wait if needed
    dad = scrape_detailed_auction_data(driver)
    cad = scrape_condensed_auction_data(driver)
    driver.quit()
    dad.extend(cad)
    print(f"Total number of auctions: {len(dad)}")
    return dad

In [15]:
# scrape_auction_data('https://hibid.com/livecatalog/481241/335--new--returned-and-over-stock-items-sunday-03-sep-2023')

In [20]:
def update_csv_with_data(filename, data):
    # If the CSV exists, load it into a DataFrame. Otherwise, create a new DataFrame
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=["link", "title", "lot_number", "number_of_bids", "currency", "max_bid"])

    for item in data:
        try:
            # Check if item already exists in the DataFrame
            existing_entry = df[df['link'] == item['link']]
            
            # If it does not exist, append it to the DataFrame
            if existing_entry.empty:
                df = pd.concat([df, pd.DataFrame([item])], ignore_index=True)
            else:
                # If it does and has a different max_bid, update the max_bid
                if existing_entry.iloc[0]['max_bid'] != item['max_bid']:
                    df.loc[df['link'] == item['link'], 'max_bid'] = item['max_bid']
        except Exception as e:
            print(f"Error updating the DataFrame: {e}")

    # Save the updated DataFrame to the CSV
    df.to_csv(filename, index=False)

In [17]:
def main(scrape_interval=60, headless=True):
    while True:
        data = scrape_auction_data("https://hibid.com/livecatalog/478452/undeliverable-freight-return-merchandise-closes-sept-3--2023", headless=headless)
        update_csv_with_data("auctions.csv", data)
        time.sleep(scrape_interval)

In [21]:
main()

C:\Users\shiva\AppData\Local\Temp\ipykernel_2096\2574675377.py:3: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service(executable_path="geckodriver/geckodriver.exe")


Number of detailed auctions: 31
Number of condensed auctions: 30
Total number of auctions: 61


C:\Users\shiva\AppData\Local\Temp\ipykernel_2096\2574675377.py:3: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service(executable_path="geckodriver/geckodriver.exe")


Number of detailed auctions: 30
Number of condensed auctions: 29
Total number of auctions: 59


C:\Users\shiva\AppData\Local\Temp\ipykernel_2096\2574675377.py:3: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service(executable_path="geckodriver/geckodriver.exe")


...

Number of detailed auctions: 45
Number of condensed auctions: 27
Total number of auctions: 72


C:\Users\shiva\AppData\Local\Temp\ipykernel_2096\2574675377.py:3: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service(executable_path="geckodriver/geckodriver.exe")


Number of detailed auctions: 42
Number of condensed auctions: 30
Total number of auctions: 72


C:\Users\shiva\AppData\Local\Temp\ipykernel_2096\2574675377.py:3: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service(executable_path="geckodriver/geckodriver.exe")


Number of detailed auctions: 29
Number of condensed auctions: 29
Total number of auctions: 58
